In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

print("")
print("train.num = ", mnist.train.num_examples)
print("test.num = ", mnist.test.num_examples)
print("validation.num = ", mnist.validation.num_examples)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

train.num =  55000
test.num =  10000
validation.num =  5000


In [2]:
learning_rate = 0.001 # 학습율
epochs = 30 # 반복횟수
batch_size = 100 #한번에 입력으로 주어지는 MNIST 개수

In [3]:
X = tf.placeholder(tf.float32, [None, 784]) # 28 x 28

T = tf.placeholder(tf.float32, [None, 10]) # 정답데이터 10개

A1 = tf.reshape(X, [-1, 28, 28, 1]) # image 28 x 28 x 1 (black / white)


In [4]:
W2 = tf.Variable(tf.random_normal([5, 5, 1, 32], stddev = 0.01)) # 3 x 3 x 32

b2 = tf.Variable(tf.constant(0.1, shape = [32]))

C2 = tf.nn.conv2d(A1, W2, strides=[1, 1, 1, 1], padding= 'SAME')

Z2 = tf.nn.relu(C2 + b2)

A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # 14 x 14 x 32

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
W3 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev = 0.01))

b3 = tf.Variable(tf.constant(0.1, shape = [64]))
# 14 x 14 x 32 => 14 x 14 x 64
C3 = tf.nn.conv2d(A2, W3, strides=[1, 1, 1, 1], padding='SAME')

Z3 = tf.nn.relu(C3 + b3)
# 14 x 14 x 64 => 7 x 7 x 64
A3 = P3 = tf.nn.max_pool(Z3, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding= 'SAME')

In [6]:
A3_flat = P3_flat = tf.reshape(A3 , [-1, 7 * 7 * 64])


In [7]:
W4 = tf.Variable(tf.random_normal([7*7*64, 10], stddev = 0.01))

b4 = tf.Variable(tf.random_normal([10]))

Z4 = logits = tf.matmul(A3_flat, W4) + b4

y = A4 = tf.nn.softmax(Z4)

In [8]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = Z4, labels = T))

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [9]:
predicted_val = tf.equal(tf.argmax(A4, 1), tf.argmax(T, 1))

accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype= tf.float32))

accuracy_index = tf.cast(predicted_val, dtype = tf.float32)

predicted_list = tf.argmax(A4, 1)

In [10]:
index_label_prediction_list = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start_time = datetime.now()
    
    for i in range(epochs):
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
            
            loss_val, _ = sess.run([loss, train], feed_dict = {X : batch_x_data, T : batch_t_data})
            
            if step % 100 == 0:
                print("epochs = ", i, "step = ", step, ", loss_val = ", loss_val)
    end_time = datetime.now()
    
    print("\n Elapsed Time : ", end_time - start_time)
    
    test_x_data = mnist.test.images
    test_t_data = mnist.test.labels
    
    accuracy_val,predicted_list_val, index_label  = sess.run([accuracy, predicted_list, accuracy_index], feed_dict = {X : test_x_data, T: test_t_data})
    print("\n accuracy = ", accuracy_val)
    print("type(accuracy_val) = ", type(accuracy_val), ', type(predicted_list_val) = ', type(predicted_list_val), ', type(index_label) = ', type(index_label))
    print("index_label.shape = ", index_label.shape)
    
    index_label_list = list(index_label)
    print("length of index_label_list = ", len(index_label_list))
    print("false label count = ", index_label_list.count([0]))
        
    # numpy type 으로 디버그
    temp_list = [] 
    
    for index in range(len(index_label)):
        
        if index_label[index] == 0:
            
            temp_list.append(index)
            temp_list.append(np.argmax(test_t_data[index]))  # one-hot encoding 이므로 argmax 로 정답 추출
            temp_list.append(predicted_list_val[index])
            
            index_label_prediction_list.append(temp_list)
            
            temp_list = []
            
    print("\nlength of index_label_false_list", len(index_label_prediction_list))

epochs =  0 step =  0 , loss_val =  3.0896268
epochs =  0 step =  100 , loss_val =  0.57769316
epochs =  0 step =  200 , loss_val =  0.15617126
epochs =  0 step =  300 , loss_val =  0.059980612
epochs =  0 step =  400 , loss_val =  0.112111494
epochs =  0 step =  500 , loss_val =  0.060650837
epochs =  1 step =  0 , loss_val =  0.091379434
epochs =  1 step =  100 , loss_val =  0.082418405
epochs =  1 step =  200 , loss_val =  0.10866155
epochs =  1 step =  300 , loss_val =  0.054345284
epochs =  1 step =  400 , loss_val =  0.15476324
epochs =  1 step =  500 , loss_val =  0.05714016
epochs =  2 step =  0 , loss_val =  0.018084418
epochs =  2 step =  100 , loss_val =  0.05269741
epochs =  2 step =  200 , loss_val =  0.033849772
epochs =  2 step =  300 , loss_val =  0.039092787
epochs =  2 step =  400 , loss_val =  0.04248695
epochs =  2 step =  500 , loss_val =  0.021023441
epochs =  3 step =  0 , loss_val =  0.03450719
epochs =  3 step =  100 , loss_val =  0.06296139
epochs =  3 step = 

KeyboardInterrupt: 

In [15]:
print(index_label_prediction_list)

FailedPreconditionError: Attempting to use uninitialized value Variable_4
	 [[node Variable_4/read (defined at <ipython-input-7-ae911b194cde>:1) ]]

Caused by op 'Variable_4/read', defined at:
  File "C:\Users\USER\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\USER\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\USER\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\USER\anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\USER\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\USER\anaconda\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\USER\anaconda\lib\asyncio\base_events.py", line 438, in run_forever
    self._run_onc

In [ ]:
# check false data
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

save_count = 0

# 현재 디렉토리 저장
curr_dir = os.getcwd()

#image 저장할 디렉토리 생성, 현재 시간으로 생성
now = datetime.now()
algorithm_name = 'CNN_1Conv_Adam_'
dir_name = algorithm_name + str(now.year) + '-' + str(now.month) + '-' +str(now.day) + '-' + str(now.hour) + str(now.minute) + str(now.second)

os.mkdir(dir_name)

# change dir
os.chdir(dir_name)

start_time = datetime.now()

for list_data in index_label_prediction_list:
    index_int = list_data[0]
    label_int = list_data[1]
    prediction_int = list_data[2]
    
    # 저장할 이미지를 인덱스를 이용하여 가져옴
    img = test_x_data[index_int].reshape(28,28)
    plt.imshow(img, cmap = 'gray')
    
    # 정답 문자열
    label_str = str(label_int)
    
    # 예측값 문자열
    prediction_str = str(prediction_int)
    
    # 정답과 오답을 나타내는 문자열
    label_prediction_str = 'label = ' + label_str + ', prediction = ' + prediction_str
    
    # 저장 파일 이름 생성, str(index_int).png
    save_image_name = str(index_int) + '.png'
    
    plt.title(label_prediction_str)
    plt.savefig(save_image_name)
    
    save_count += 1
    
    if save_count % 10 == 0:
        
        print(save_count, 'image is saved now')
        
end_time = datetime.now()

print('Elapsed Time = ', end_time - start_time)
print('Total ', sabe_count, " data is saved")

# 원래 dir 로 복귀
os.chdir(curr_dir)